# 第5章: 係り受け解析

In [1]:
import re
import CaboCha

In [2]:
cabocha = CaboCha.Parser()
space_pattern = re.compile(r'\u3000')
chapter_pattern = re.compile(r'([\n|\r\n|\r]|(?<=^))([一二三四五六七八九十]+)[\n|\r\n|\r]')

with open("../data/neko.txt") as fr,\
     open("../data/neko.txt.cabocha", "w") as fw:
    text = fr.read()
    text = re.sub(space_pattern, '', text)
    text = re.sub(chapter_pattern, '', text)
    
    for sentence in text.split('。'):
        sentence = sentence + '。'
        tree = cabocha.parse(sentence)
        print(tree.toString(CaboCha.FORMAT_LATTICE), file=fw)

## 40. 係り受け解析結果の読み込み（形態素）

In [55]:
class Morph(object):
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
        self.info = {
            'surface': self.surface,
            'base': self.base,
            'pos': self.pos,
            'pos1': self.pos1,
        }
    
    def as_dict(self):
        return self.info
    
    def __str__(self):
        return str(self.info)
    
    def __repr__(self):
        return str(self.info)
    
    def __getitem__(self, key):
        return self.info[key]

In [56]:
neko_morphs = []
with open('../data/neko.txt.cabocha') as fr:
    sentence_morphs = []
    for line in fr:
        line = line.rstrip()
        
        if line == 'EOS':
            neko_morphs.append(sentence_morphs)
            sentence_morphs = []
            continue
        
        if len(line) == 0\
        or line[0] == '*':
            continue
        
        m = re.split('[\t,]', line)
        m = Morph(m[0], m[-3], m[1], m[2])
        sentence_morphs.append(m)

In [57]:
neko_morphs[2]

[{'surface': 'どこ', 'base': 'どこ', 'pos': '名詞', 'pos1': '代名詞'},
 {'surface': 'で', 'base': 'で', 'pos': '助詞', 'pos1': '格助詞'},
 {'surface': '生れ', 'base': '生れる', 'pos': '動詞', 'pos1': '自立'},
 {'surface': 'た', 'base': 'た', 'pos': '助動詞', 'pos1': '*'},
 {'surface': 'か', 'base': 'か', 'pos': '助詞', 'pos1': '副助詞／並立助詞／終助詞'},
 {'surface': 'とんと', 'base': 'とんと', 'pos': '副詞', 'pos1': '一般'},
 {'surface': '見当', 'base': '見当', 'pos': '名詞', 'pos1': 'サ変接続'},
 {'surface': 'が', 'base': 'が', 'pos': '助詞', 'pos1': '格助詞'},
 {'surface': 'つか', 'base': 'つく', 'pos': '動詞', 'pos1': '自立'},
 {'surface': 'ぬ', 'base': 'ぬ', 'pos': '助動詞', 'pos1': '*'},
 {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}]

## 41. 係り受け解析結果の読み込み（文節・係り受け）

In [98]:
class Chunk(object):
    def __init__(self, dst):
        self.morphs = []
        self.dst = dst
        self.srcs = []
        self.info = {
            'morphs': self.morphs,
            'dst': self.dst,
            'srcs': self.srcs,
        }
        
    def get_text(self, ignore_kv=None):
        if ignore_kv:
            k, v = ignore_kv
            return ''.join([
                m['surface'] for m in self.morphs
                if m[k] != v
            ])
        else:
            return ''.join([
                m['surface'] for m in self.morphs
            ])
        
    def search_kv(self, kv):
        k, v = kv
        result = [
            m for m in self.morphs
            if m[k] == v
        ]
        if len(result) == 0:
            return None
        else:
            return result
    
    def as_dict(self):
        return self.info
    
    def __str__(self):
        return str(self.info)
    
    def __repr__(self):
        return str(self.info)
    
    def __len__(self):
        return len(self.morphs)
    
    def __getitem__(self, key):
        return self.info[key]

In [99]:
neko_chunks = []
with open("../data/neko.txt.cabocha") as fr:
    sentence_chunks = []
    id_to_srcs = {}
    
    for line in fr:
        line = line.rstrip()
        if len(line) == 0:
            continue
        
        elif line == "EOS":
            for chunk_id, srcs in id_to_srcs.items():
                if chunk_id == -1:
                    continue
                sentence_chunks[chunk_id].srcs += srcs
            neko_chunks.append(sentence_chunks)
            sentence_chunks = []
            id_to_srcs = {}
            
        elif line[0] == "*":
            line = line.split()
            chunk_id = int(line[1])
            dst = int(line[2][:-1])
            
            sentence_chunks.append(Chunk(dst))
            
            if dst in id_to_srcs.keys():
                id_to_srcs[dst].append(chunk_id)
            else:
                id_to_srcs[dst] = [chunk_id]
        
        else:
            m = re.split('[\t,]', line)
            m = Morph(m[0], m[-3], m[1], m[2])
            sentence_chunks[-1].morphs.append(m)

In [100]:
for c in neko_chunks[7]:
    print(c.get_text(), c.dst)

しかし 9
その 2
当時は 5
何という 4
考も 5
なかったから 9
別段 7
恐し 9
いとも 9
思わなかった。 -1


## 42. 係り元と係り先の文節の表示

In [101]:
all_dependencies = []
for sentence_chunks in neko_chunks:
    for c in sentence_chunks:
        if c['dst'] == -1:
            continue
        src = c.get_text(ignore_kv=('pos', '記号'))
        dst = sentence_chunks[c['dst']].get_text(ignore_kv=('pos', '記号'))
        src_dst = src + '\t' + dst
        all_dependencies.append(src_dst)

In [102]:
print(len(all_dependencies))
print('\n'.join(all_dependencies[:20]))

73144
吾輩は	猫である
名前は	無い
まだ	無い
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ
何でも	薄暗い
薄暗い	所で
じめじめした	所で
所で	泣いて
ニャーニャー	泣いて
泣いて	記憶している
いた事だけは	記憶している
吾輩は	見た
ここで	始めて
始めて	人間という
人間という	ものを
ものを	見た
しかも	種族であったそうだ


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

In [108]:
nv_dependencies = []
for sentence_chunks in neko_chunks:
    for c in sentence_chunks:
        if c['dst'] == -1:
            continue
        
        if c.search_kv(('pos', '名詞')):
            dst_chunk = sentence_chunks[c['dst']]
            
            if dst_chunk.search_kv(('pos', '動詞')):
                src = c.get_text(ignore_kv=('pos', '記号'))
                dst = dst_chunk.get_text(ignore_kv=('pos', '記号'))
                src_and_dst = src + '\t' + dst
                nv_dependencies.append(src_and_dst)

In [109]:
print(len(nv_dependencies))
print('\n'.join(nv_dependencies[:20]))

29528
どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて
掌に	載せられて
スーと	持ち上げられた
時	フワフワした
感じが	あったばかりである
上で	落ちついて
顔を	見たのが
ものの	見始であろう
ものだと	思った
感じが	残っている
今でも	残っている


## 44. 係り受け木の可視化